Fist, I imprort all the necessary libraries and read the IMS raw data.

In [1]:
#Import the libraries

import pandas as pd
import numpy as np
import re

In [2]:
# Read the raw data

df = pd.read_pickle('IMS-Raw.pkl')

Next, I generate the matrix "DrugHistory" to save all the information about history of drug development. Then, the code reads the lines for the drug development history and extract the data for Preclinical study, Phase I, Phase II and Phase III clinical trials. If the date for each of these development stages is not mentioned, the code would write "NaT" for that date. After that, the code will add the date information to the main data set and clean it. 

In [3]:
#generate a matrix for drug history

Dh = df.DH.str.split(pat ='\n' , expand = True)

columns={i:'dh'+str(i) for i in range(Dh.shape[1])}
Dh=Dh.rename(columns=columns)

Dh_col = [col for col in Dh if col.startswith('dh')]

DrugHistory = pd.DataFrame()
for col in Dh_col:
    m=Dh.dropna(subset=[col])[[col]].rename(columns={col:'DrugHistory'}).reset_index(drop=True)
    DrugHistory=DrugHistory.append(m,ignore_index=True,sort=False)

    
#Extract the date of drug development stages

df=df.dropna(subset=['DH']).reset_index(drop=True)

Date = pd.DataFrame(columns=['Preclinical','PhaseI', 'PhaseII', 'PhaseIII'], index=range(df.shape[0]))

for i in range(len(df.DH)):

    items=re.findall("^.*phase I.*$|^.*Phase I.*$|^.*preclinical.*$|^.*Preclinical.*$",df.DH[i],re.MULTILINE)
    Fil = re.compile("^.*DATE UNKNOWN.*$|^.*trial planned.*$")
    items = [it for it in items if not Fil.match(it)]
    
    FilPre = re.compile("^.*Preclinical.*$", re.IGNORECASE)
    Pre = list(filter(FilPre.match, items)) 
    if not Pre==[]:
        Date.Preclinical[i]=Pre[-1]
    
    FilPhI = re.compile("^.*Phase I(?!I).*$", re.IGNORECASE)
    PhI = list(filter(FilPhI.match, items)) 
    if not PhI==[]:
        Date.PhaseI[i]=PhI[-1]
        
    FilPhII = re.compile("^.*Phase II(?!I).*$|^.*Phase I/II.*$", re.IGNORECASE)
    PhII = list(filter(FilPhII.match, items)) 
    if not PhII==[]:
        Date.PhaseII[i]=PhII[-1]
        
    FilPhIII = re.compile("^.*Phase III.*$|^.*Phase II/III.*$", re.IGNORECASE)
    PhIII = list(filter(FilPhIII.match, items)) 
    if not PhIII==[]:
        Date.PhaseIII[i]=PhIII[-1]
        
for phase in list(Date.columns):
    dt=Date[phase].str.split(r"\:| ", expand=True,n=2)
    dt[0]=pd.to_datetime(dt[0].str.slice(stop=4), errors='coerce')
    dt[1]=pd.to_datetime(dt[1].str.slice(stop=4), errors='coerce')
    dt[0].update(dt.pop(1))
    Date[phase]=dt[0]
    
    
#Add the date information into the main data set 

df=pd.concat([df,Date], axis=1)


#Clean the data set

df=df[~(pd.isna(df.Preclinical) & pd.isna(df.PhaseI) & pd.isna(df.PhaseII) & pd.isna(df.PhaseIII))]

df=df.rename(columns={"DN": "DrugName","CC":"EphMRA","CI":"CompanyInformation","IC":"Indication","DH":"DrugHistory", "HP": "HighestPhase"})
df=df.reset_index(drop=True)

df=df.drop(columns=['DrugHistory'], axis=1)

df=df.drop_duplicates().reset_index(drop=True)

df['DrugID'] = df.index

For the next step, I clean the ICD (EphMRA) codes assigned to each drug. First, I read the line for ICD information of each drug and assign each code to its drug in a separate line. Then, for consistency, I trim the ICD codes to their first three digits. For example, the code "J5D2" will go to "J5D". The result is saved in a new dataframe "Df".

In [4]:
#Assign ICD codes to the drugs in seperate lines

CC=df.EphMRA
CC = CC.str.split(pat ='\n' , expand = True)
for i in range(CC.shape[1]):
    CC[i] = CC[i].str.split(n = 1, expand = True)[0]
    
columns={i:'CC'+str(i) for i in range(CC.shape[1])}
CC=CC.rename(columns=columns)

dfcc=pd.concat([df,CC], axis=1)
dfcc=dfcc.drop(columns=['EphMRA'])
CC_col = [col for col in dfcc if col.startswith('CC')]

Df = pd.DataFrame()
for cc in CC_col:
    m=dfcc.dropna(subset=[cc])[['DrugID','CompanyInformation',cc,'Preclinical','PhaseI','PhaseII','PhaseIII']].rename(columns={cc:'EphMRA'}).reset_index(drop=True)
    Df=Df.append(m,ignore_index=True,sort=False)

#Trim the ICD codes
for i in range(len(Df)):
    s=Df.EphMRA[i]
    m = re.match(r"([a-zA-Z]+)([0-9]+)([a-zA-Z]+)",s).group()
    Df.loc[i,'EphMRA']=m

In the end, I extract the information from all the firms who were involved in developing the drugs. To do that, first, I read the company information section of IMS data and assign the drugs to each of the firms involved separately. Then the code prints all the possible roles each firm had in drug development. Next, I delete all the data point where the firm was only the licensee of the drug. After that, I clean the data and save it in the file "IMS.pkl".

In [5]:
#Assign drugs to rach firm seperately

CI = Df.CompanyInformation.str.split(pat ='\n' , expand = True)
for i in range(CI.shape[1]):
    CI[i] = CI[i].str.split(pat =',',n = 1, expand = True)[0]
    
columns={i:'ci'+str(i) for i in range(CI.shape[1])}
CI =CI.rename(columns=columns)
Dfci=pd.concat([Df,CI], axis=1)
Dfci=Dfci.drop(columns=['CompanyInformation'])

data = pd.DataFrame()
ci_col = [col for col in Dfci if col.startswith('ci')]
for ci in ci_col:
    m=Dfci.dropna(subset=[ci])[['DrugID','EphMRA',ci,'Preclinical','PhaseI','PhaseII','PhaseIII']].rename(columns={ci:'Company'}).reset_index(drop=True)
    data=data.append(m,ignore_index=True,sort=False)

In [6]:
#Print all the possible roles of the firms in drug development

company=data.Company
company=company.str.split(pat =':' , expand = True)
Prefix=company[company[1].notnull()][0].unique()
print(Prefix)

['Co-developer' 'Originator' 'Developer' 'Licensee']


In [7]:
#Delete the data points where the firm was only a licensee

data=data[~data.Company.str.startswith('Licensee')]


#Clean the data and save it

for pr in Prefix:
    data.Company=data.Company.str.replace(pr+":", "")
    
data.Company=data.Company.str.lstrip()

data=data.sort_values(["DrugID"], axis=0)

data=data.drop_duplicates().reset_index(drop=True)

In [8]:
data.to_pickle('IMS.pkl')

data.to_stata('IMS.dta', write_index=False, convert_dates={'Preclinical' : 'ty','PhaseI' : 'ty','PhaseII' : 'ty','PhaseIII' : 'ty'})

In [9]:
data 

,DrugID,EphMRA,Company,Preclinical,PhaseI,PhaseII,PhaseIII
0,0,J5D,Janssen,NaT,2019-01-01,NaT,NaT
1,1,L1X,Yisheng Biopharma,2019-01-01,NaT,NaT,NaT
2,2,J7E,Janssen,2019-01-01,NaT,NaT,NaT
3,3,J8X,Nanjing Sanhome,2019-01-01,NaT,NaT,NaT
4,4,J5D,Nanjing Sanhome,NaT,2018-01-01,NaT,NaT
...,...,...,...,...,...,...,...
47199,34981,C9B,Merck & Co,NaT,NaT,NaT,1993-01-01
47200,34982,N2B,MiMedx,NaT,NaT,2015-01-01,2018-01-01
47201,34982,D3A,MiMedx,NaT,NaT,2015-01-01,2018-01-01
47202,34983,D3A,Amryt Pharma,NaT,NaT,2008-01-01,2012-01-01
